# 📘 AI Fibonacci Jackpot Reversal Detector
Nifty Futures | Daily Time Frame | Real-Time Telegram Alerts
---
- Uses yfinance fallback data with proper formatting
- Detects signals near Fibonacci 0.618
- Sends Telegram alerts using GitHub secrets
- Avoids repeat alerts using a daily signal log

In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
import os
import yfinance as yf
import requests
from datetime import datetime


In [ ]:
# 🗃️ Load data (fallback to yfinance with proper formatting)
try:
    df = pd.read_csv("sample_nifty_futures_data.csv")
    print("✅ Loaded uploaded CSV")
except:
    df = yf.download("^NSEI", period="6mo", interval="1d", progress=False)
    df = df.reset_index()
    df["Symbol"] = "NIFTY"
    df.columns = [col.capitalize() for col in df.columns]  # Ensure 'Close', 'High', 'Low'
    print("📥 Using fallback yfinance data")

df["Date"] = pd.to_datetime(df["Date"])


In [ ]:
# 🔢 Fibonacci 0.618 Detection (Dummy Logic)
df["fib_match"] = df["Low"] < (df["High"].rolling(10).max() - (df["High"].rolling(10).max() - df["Low"].rolling(10).min()) * 0.618)


In [ ]:
# 🧠 Dummy AI Confidence (Simulated)
df["ai_confidence"] = np.random.uniform(0.6, 0.95, len(df))


In [ ]:
# 🧾 Signal Log to Prevent Repeat Alerts
log_file = "signal_log.csv"
today_str = datetime.today().strftime("%Y-%m-%d")
if not os.path.exists(log_file):
    pd.DataFrame(columns=["date", "symbol"]).to_csv(log_file, index=False)
signal_log = pd.read_csv(log_file)


In [ ]:
# 📬 Telegram Setup
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")

def send_telegram(msg):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {'chat_id': TELEGRAM_CHAT_ID, 'text': msg}
    r = requests.post(url, data=payload)
    print("📤 Sent to Telegram:", msg if r.status_code == 200 else r.text)


In [ ]:
# 📊 Alert Logic
for i, row in df.iterrows():
    symbol = row.get("Symbol", "NIFTY")
    if (row["fib_match"]) and (row["ai_confidence"] > 0.75):
        already_sent = ((signal_log["symbol"] == symbol) & (signal_log["date"] == today_str)).any()
        if not already_sent:
            msg = f"🚀 Jackpot Signal for {symbol} on {row['Date'].date()}\nConfidence: {round(row['ai_confidence']*100, 2)}%\nClose: {row['Close']}"
            send_telegram(msg)
            signal_log = pd.concat([signal_log, pd.DataFrame([{"date": today_str, "symbol": symbol}])], ignore_index=True)
            signal_log.to_csv(log_file, index=False)
